# Assignment 2

In [19]:
from cmath import phase

from IPython.core.magic_arguments import argument
from numpy.ma.core import angle

from toolbox import Tools
from scipy.optimize import fsolve
import numpy as np
from sympy import *

tools = Tools() # Instance of the Tools class

## Power
Calculate power into branches 1 and 2 from turbines

In [20]:
# code to calculate power
# powers = (P1, P2) ...

## Solver
Haven't actually tried this yet.

In [21]:


init_guess = np.array([5e3, 5e3, 5e3, 5e3])  # Initial guess for the solver to start from
powers = (3e6, 3e6)                 # Power parameters to put into the equations
sol = fsolve(func=tools.equations, x0=init_guess, args=powers)

P1 = powers[0]
P2 = powers[1]

V_POC = 4000

V_T1 = sol[0] + 1j * sol[1]
V_T2 = sol[2] + 1j * sol[3]
print(abs(V_T1), "@", phase(V_T1) * 180 / np.pi)

print(abs(V_T2), "@", phase(V_T2) * 180 / np.pi)

#print(abs(sol[4] + 1j * sol[5]), "@", phase(sol[4] + 1j * sol[5]) * 180 / np.pi)

4095.7768710462606 @ 0.8309102674179976
4099.598773095083 @ 1.1224938913452616


# Test Solution
### $V_B$

In [22]:
Z11, Z12, Z21, Z22, Z31, Z32, Z41, Z42, Z5, Z6 = tools.get_impedances(verb=True)

zero1 = (-P1*V_T2*Z11*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5) + V_T1**2*V_T2*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5) - V_T2*Z21*Z32*(P1*Z11 + P1*Z31 - V_T1**2)*(Z41*Z42 + Z41*Z5 + Z42*Z5) - Z41*Z42*(P1*V_T2*Z11*Z32*Z5 + P1*V_T2*Z31*Z32*Z5 + P2*V_T1*Z12*Z31*Z5 + P2*V_T1*Z31*Z32*Z5 + V_POC*V_T1*V_T2*Z31*Z32 - V_T1**2*V_T2*Z32*Z5 - V_T1*V_T2**2*Z31*Z5))/(V_T1*V_T2*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5))

zero2 = (-P2*V_T1*Z12*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5) + V_T1*V_T2**2*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5) - V_T1*Z22*Z31*(P2*Z12 + P2*Z32 - V_T2**2)*(Z41*Z42 + Z41*Z5 + Z42*Z5) - Z41*Z42*(P1*V_T2*Z11*Z32*Z5 + P1*V_T2*Z31*Z32*Z5 + P2*V_T1*Z12*Z31*Z5 + P2*V_T1*Z31*Z32*Z5 + V_POC*V_T1*V_T2*Z31*Z32 - V_T1**2*V_T2*Z32*Z5 - V_T1*V_T2**2*Z31*Z5))/(V_T1*V_T2*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5))

VB1 = -P1*Z11/V_T1 + V_T1 - Z21*(P1/V_T1 - (-P1*Z11/V_T1 + V_T1)/Z31)
VB2 = -P2*Z12/V_T2 + V_T2 - Z22*(P2/V_T2 - (-P2*Z12/V_T2 + V_T2)/Z32)
VB3 = Z41*Z42*(P1*V_T2*Z11*Z32*Z5 + P1*V_T2*Z31*Z32*Z5 + P2*V_T1*Z12*Z31*Z5 + P2*V_T1*Z31*Z32*Z5 + V_POC*V_T1*V_T2*Z31*Z32 - V_T1**2*V_T2*Z32*Z5 - V_T1*V_T2**2*Z31*Z5)/(V_T1*V_T2*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5))

# VPOC as seen from VB
IT1 = P1/V_T1 - VB1/Z41 - (-P1*Z11/V_T1 + V_T1)/Z31
IT2 = P2/V_T2 - VB1/Z42 - (-P2*Z12/V_T2 + V_T2)/Z32
V_POCx = VB1 - Z5 * (IT1 + IT2)

# Generator current I11
I21 = IT1 + VB1 / Z41
I11 = I21 + (VB1 + I21 * Z21) / Z31
P1x = I11 * V_T1

print("zero1:", zero1)
print("zero2:", zero2)
print("VB1:", VB1)
print("VB2:", VB2)
print("VB3:", VB3)
print("VPOCx:", V_POCx)
print("P1:", P1x)





Z11 :  (0.042+0.00314j)
Z12 :  (0.042+0.00314j)
Z21 :  (0.05082+0.01976j)
Z22 :  (0.06551+0.04745j)
Z31 :  (4.94066+15.72661j)
Z32 :  (4.94066+15.72661j)
Z41 :  -25.98183j
Z42 :  -9.74319j
Z5 :  (0.02451+0.02767j)
Z6 :  -0.70151j
zero1: (-3.119848728021732e-08-7.922606682150992e-09j)
zero2: (2.3646991281692424e-08+7.713462571611963e-09j)
VB1: (4035.6754272305593+33.23434152549724j)
VB2: (4035.6754272854037+33.23434154113261j)
VB3: (4035.6754272617563+33.23434153341904j)
VPOCx: (3999.999999968952-7.998643525297666e-09j)
P1: (3000000-2.1827872842550278e-11j)


## Some Mesh Equations

In [23]:
ia, ib, ic, iD = symbols('ia ib ic iD')

eq1 = -V_T1 + ia * Z11 + (ia - ib) * Z31
eq2 = (ib - ia) * Z31 + ib * Z21 + (ib - ic) * Z41
eq3 = (ic - ib) * Z41 + ic * Z5 + (ic - iD) * Z6
eq4 = (iD - ic) * Z6 + V_POC

sol = solve([eq1, eq2, eq3, eq4], [ia, ib, ic, iD])
sol

{iD: 852.202548718301 - 5603.60198479583*I,
 ia: 928.177218469403 + 19.2853137811517*I,
 ib: 851.202163842111 + 253.036625949187*I,
 ic: 852.202548718301 + 98.3837317299528*I}

In [24]:
sol = list(sol.values())


In [25]:
ia = 928.177218469403 + 19.2853137811517j
ib = 851.202163842111 + 253.036625949187
V_T1 - ia * Z11 - ib * Z21

(4000.3058815526188+33.85114738971825j)

In [26]:
V_T1 * ia

(3800061.571603371+134109.46136265356j)

In [27]:
type(x)

NameError: name 'x' is not defined